[MetaWeather](https://www.metaweather.com) - это автоматический агрегатор данных о погоде, который принимает прогнозы погоды от различных прогнозистов и вычисляет наиболее вероятный результат.

Идея состоит в том, что объединенные знания прогнозистов произведут лучшее предсказание.

**История**

Я (Джейсон) собрался на свадьбу друга, которая была частично на улице в непредсказуемой Великобритании. При проверке прогноза погоды я задавался вопросом, должен ли я проверить другого прогнозиста, чтобы увидеть, что он предсказал. Этот процесс привел меня к проверке некоторых сайтов. Пара плохо написанных PHP-скриптов, неуклюжая база данных MySQL и несколько часов спустя я перебирал данные о погоде с нескольких сайтов.

Я приступил к созданию внешнего интерфейса и с тех пор собрал около миллиарда точек прогнозов и данных наблюдений по состоянию на начало 2017 года. 

Далее пример работы с API сайта:

In [1]:
import requests
import pandas as pd
from pprint import pprint as pp

Пример получения прогноза погоды в Санкт-Петербурге на 24.09.2019

Для начала узнаем ID города:

In [2]:
json_city = requests.get("https://www.metaweather.com/api/location/search/?query=St Petersburg").json()

In [3]:
pp(json_city)

[{'latt_long': '59.932739,30.306721',
  'location_type': 'City',
  'title': 'St Petersburg',
  'woeid': 2123260}]


In [4]:
ident = json_city[0]["woeid"]

In [5]:
print(ident)

2123260


Запрос погоды для определенного дня возвращает историю прогноза, т.е. каким образом менялся прогноз за несколько дней.   
Поле predictability показывает вероятность прогноза.

In [6]:
pp(requests.get("https://www.metaweather.com/api/location/"+ str(ident)+"/2019/10/1/").json()[:3])

[{'air_pressure': 1015.96,
  'applicable_date': '2019-10-01',
  'created': '2019-09-24T07:20:28.915907Z',
  'humidity': 71,
  'id': 4673607827456000,
  'max_temp': 21.03,
  'min_temp': 16.605,
  'predictability': 77,
  'the_temp': 28.32,
  'visibility': 9.859918078422016,
  'weather_state_abbr': 'hr',
  'weather_state_name': 'Heavy Rain',
  'wind_direction': 27.5,
  'wind_direction_compass': 'NNE',
  'wind_speed': 4.983335321721149},
 {'air_pressure': 1015.96,
  'applicable_date': '2019-10-01',
  'created': '2019-09-24T04:20:28.690001Z',
  'humidity': 71,
  'id': 5490819576365056,
  'max_temp': 21.03,
  'min_temp': 16.605,
  'predictability': 77,
  'the_temp': 28.32,
  'visibility': 9.859918078422016,
  'weather_state_abbr': 'hr',
  'weather_state_name': 'Heavy Rain',
  'wind_direction': 27.5,
  'wind_direction_compass': 'NNE',
  'wind_speed': 4.983335321721149},
 {'air_pressure': 1014.67,
  'applicable_date': '2019-10-01',
  'created': '2019-09-24T01:20:29.025872Z',
  'humidity': 75,


Данные на сайте собираются из разных источников. Можно посмотреть такие параметры, как атмосферное давление, максимальная и минимальная температура, скорость ветра, влажность, видимость и вероятность (предсказуемость) данного прогноза.

In [111]:
pd.DataFrame(requests.get("https://www.metaweather.com/api/location/"+ str(ident)+"/2019/9/27/").json()).head()

,air_pressure,applicable_date,created,humidity,id,max_temp,min_temp,predictability,the_temp,visibility,weather_state_abbr,weather_state_name,wind_direction,wind_direction_compass,wind_speed
0,1015.975,2019-09-27,2019-09-24T07:20:16.956506Z,74,5834806376005632,20.675,15.195,71,18.950,11.727760,hc,Heavy Cloud,329.510735,NNW,3.423494
1,1015.975,2019-09-27,2019-09-24T04:20:16.792092Z,74,4845987279405056,20.675,15.195,71,18.885,10.502105,hc,Heavy Cloud,321.989274,NW,3.453320
2,1015.545,2019-09-27,2019-09-24T01:20:16.974681Z,71,5098331288109056,20.745,15.615,70,19.445,11.183439,lc,Light Cloud,266.931728,W,4.015569
3,1016.545,2019-09-27,2019-09-23T22:20:17.198340Z,71,6464681248030720,20.745,15.615,70,20.050,12.841568,lc,Light Cloud,321.906990,NW,3.598008
4,1016.860,2019-09-27,2019-09-23T19:20:19.981937Z,72,4831075152953344,20.845,15.845,70,20.095,13.148525,lc,Light Cloud,280.500000,W,3.486162


## Задание

Определите c помощью возможностей модуля *pandas* для города Санкт-Петербург в период с 24 по 27 сентября среднюю наиболее вероятную минимальную температуру.

In [79]:
f = (pd.DataFrame(requests.get("https://www.metaweather.com/api/location/"+ str(ident)+"/2019/9/26/").json()).head())[['predictability','min_temp']].head()
max(f['predictability'])

71

In [112]:
import numpy
s = []
for i in range(24,28):
    f = (pd.DataFrame(requests.get("https://www.metaweather.com/api/location/"+ str(ident)+f"/2019/9/{i}/").json()).head())[['min_temp','predictability']].head()
    p = (f[f.predictability == max(f['predictability'])])
    s.append(numpy.mean(p['min_temp']))
numpy.mean(s)

13.9475